In [1]:
import sys

# https://github.com/minrk/findspark
import findspark
findspark.init()

import pyspark

from pyspark import SparkContext # pip3 install pyspark
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors
from pymongo import MongoClient

sys.path.append("..")
import config # ../config.py

In [2]:
# Adapted from:
# https://github.com/apache/spark/blob/master/examples/src/main/python/mllib/latent_dirichlet_allocation_example.py

In [3]:
client = MongoClient(host=config.mongodb_ip, port=config.mongodb_port)
db = client['steam']
collection = db['apps']

print(config.mongodb_ip)

192.168.1.171


In [4]:
sc.stop()

NameError: name 'sc' is not defined

In [5]:
#sc = SparkContext(master="spark://192.168.1.145:7077", appName="SteamFindTopicsLDA")

# leverage findspark package
sc = pyspark.SparkContext(appName="SteamFindTopicsLDA")

In [6]:
# Load and parse the data
ret = collection.find({"updated_date": {"$exists": True}, "type": {"$in": ["game", "dlc"]}},
                        {"detailed_description":1})#.limit(500)
descriptions = []
for v in ret:
    descriptions.append(v['detailed_description'])

In [7]:
data = sc.parallelize(descriptions)

In [8]:
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.IllegalArgumentException: Unsupported class file major version 56
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:166)
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:148)
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:136)
	at org.apache.xbean.asm6.ClassReader.<init>(ClassReader.java:237)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:49)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:517)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:500)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:134)
	at scala.collection.mutable.HashMap$$anon$1$$a

In [ ]:
# Cluster the documents into k topics using LDA
ldaModel = LDA.train(corpus, k=50, maxIterations=100)

In [ ]:
# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

In [ ]:
# Save and load model
ldaModel.save(sc, "/data/spark-model")
sameModel = LDAModel.load(sc, "/data/spark-model")

In [ ]:
sc.stop()